### Day 4 Exercise: Data Warehousing, Data Modeling & DevOps/CICD 🏗️

#### Objective
This exercise focuses on the principles of data warehousing and analytical data modeling. You will design a Star Schema for our fraud analytics use case and conceptualize how to build and deploy the data pipeline using DevOps and CI/CD practices.

#### Scenario
The fraud detection rules are now implemented in PySpark. The next step is to store the enriched transaction data and fraud flags in a structured, optimized way for business intelligence (BI) and analytics. You need to design a data model that allows analysts to easily query and build dashboards on fraudulent activities. You also need to think about how to automate the deployment of your data pipeline.

#### Data to Use 📊
You will use the conceptual output from the Day 3 exercise: a PySpark DataFrame (`sdf_final_transactions`) containing enriched transaction data along with the boolean fraud flags (`is_fraudulent_rule1`, `is_fraudulent_rule2`).

---

### Part 1: Data Modeling for Fraud Analytics (Conceptual) ⭐

#### 1.1 Design a Star Schema
Based on the available data, design a **Star Schema** to model the data for analytical querying. Define the tables, columns, data types, and relationships.

* **Fact Table**: `fact_transactions`
    * What are the measures (the quantitative values) in this table?
    * What are the foreign keys that will link to the dimension tables?
    * Define all columns, their data types (e.g., INTEGER, DECIMAL, VARCHAR, DATETIME, BOOLEAN), and their purpose.

* **Dimension Tables**:
    * **`dim_customer`**:
        * How would you populate this table?
        * Define its columns, including a surrogate key.
    * **`dim_date`**:
        * Why is a separate date dimension useful?
        * What are some useful attributes this table would contain (e.g., day, week, month, quarter, year, day_of_week)?
        * Define its columns.
    * **`dim_location`** (Optional, but recommended):
        * How could you create a location dimension from the `ip_address`? (Conceptually - e.g., using a Geo-IP lookup service).
        * What columns would it have?

#### 1.2 Slowly Changing Dimensions (SCD)
* In Day 2, we discussed SCD Type 2 for `customer_tier`. Now, provide a more detailed explanation.
* **Task**: Illustrate how you would update the `dim_customer` table if "Alice Smith" (customer_id 'C101') was upgraded from "Gold" to "Platinum" tier on `2024-07-18`.
* Show the state of the `dim_customer` table **before** and **after** the change, including the necessary columns to handle SCD Type 2 (e.g., `customer_sk`, `customer_id`, `tier`, `start_date`, `end_date`, `is_current`).

---

### Part 2: DevOps & CI/CD for Data Pipelines (Conceptual) 🤖

#### 2.1 Version Control with Git
* Describe how you would structure your project in a Git repository. What would be the key folders and files? (e.g., `/src` for code, `/tests` for unit tests, `/notebooks` for exploration, `requirements.txt`, etc.).
* Explain the purpose of a `.gitignore` file in this project. What are some specific files or directories you would add to it?

#### 2.2 Continuous Integration (CI)
* What is the primary goal of Continuous Integration for this data engineering project?
* Describe a simple CI pipeline for your PySpark fraud detection script. What are the key stages or steps?
    * **Hint**: Think about what should happen automatically when a developer pushes a change to a feature branch. (e.g., Trigger, Linting, Unit Testing, Packaging).

#### 2.3 Continuous Deployment/Delivery (CD)
* What is the difference between Continuous Delivery and Continuous Deployment in the context of this project?
* Describe a conceptual CD pipeline that takes the tested PySpark application from the CI pipeline and deploys it to your Azure Databricks environment. What are the key stages?
    * **Hint**: Think about environments (Dev, Staging, Prod), approvals, and the actual deployment steps.

SOLUTION

In [ ]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("FraudDataLoading").getOrCreate()

# Define the file path
file_path = "/data/sdf_final_transaction/part-00000-b3362811-915c-4425-8025-3a497ac441f9-c000.csv"

# Load the dataset into a DataFrame
# We assume the CSV has a header and infer the schema
sdf_final_transactions = spark.read.csv(file_path, header=True, inferSchema=True)


DataFrame Schema:
root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- currency: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- transaction_hour: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- customer_tier: string (nullable = true)
 |-- last_login_date: string (nullable = true)
 |-- is_fraudulent_rule1: boolean (nullable = true)
 |-- timestamp_to_date: date (nullable = true)
 |-- is_fraudulent_rule1_spark: boolean (nullable = true)
 |-- prev_ip_address: string (nullable = true)
 |-- transaction_count_10min: integer (nullable = true)
 |-- is_fraudulent_rule2: boolean (nullable = true)
 |-- is_fraudulent_combined: boolean (nullable = true)


DataFrame Records:
+--------------+-----------+------+-------------------+--

In [ ]:

# Print the schema
print("DataFrame Schema:")
sdf_final_transactions.printSchema()

# Show some records
print("\nDataFrame Records:")
sdf_final_transactions.show()

DataFrame Schema:
root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- currency: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- transaction_hour: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- customer_tier: string (nullable = true)
 |-- last_login_date: string (nullable = true)
 |-- is_fraudulent_rule1: boolean (nullable = true)
 |-- timestamp_to_date: date (nullable = true)
 |-- is_fraudulent_rule1_spark: boolean (nullable = true)
 |-- prev_ip_address: string (nullable = true)
 |-- transaction_count_10min: integer (nullable = true)
 |-- is_fraudulent_rule2: boolean (nullable = true)
 |-- is_fraudulent_combined: boolean (nullable = true)


DataFrame Records:
+--------------+-----------+------+-------------------+--

So, I our STAR Schema we shall take in considerations these key aspects:
1. we are going to have a fact table, which works as a center of our star
2. the fact table will contain as foreign key, the primary keys from the dimensional tables
3. we have to set up constraints on the keys as type and nullable

Let's start from the fact table and lets consider the transaction table as our fact table


note: things to check
1. varchar for variables such as ip_address, city, state, long, lat shall be adapted to have an optimized ddl


```
CREATE TABLE fact_transactions (
    transaction_id UUID PRIMARY KEY NOT NULL,
    transaction_timestamp TIMESTAMP,
    customer_id UUID,
    date DATETIME FOREIGN KEY,
    location_id INT FOREIGN KEY,
    amount FLOAT,
    ip_address VARCHAR(100)
    fraud_flag BOOLEAN
);
```

As we can see from this schema, we have the primary key and the three foreign keys of the dimensional tables `dim_customer`, `dim_location`, `dim_date`. Lets move on writing the DDL for the location and date table:


```
CREATE TABLE dim_location (
    location_id UUID PRIMARY KEY NOT NULL,
    city VARCHAR(50),
    state VARCHAR(50),
    longitude VARCHAR(50),
    latitude VARCHAR(50)
);
```

`dim_location` is a useful table because let us add information thanks to joins instead of performing operations to extract it.
PRO: informations that will not change (locations are immutable or so) and easy to query. CON: another table to take in consideration in our DDL.


```
CREATE TABLE dim_date (
    date DATETIME PRIMARY KEY NOT NULL,
    day VARCHAR(10),
    week VARCHAR(10),
    month VARCHAR(10),
    quarter VARCHAR(10),
    year VARCHAR(10),
    day_of_week VARCHAR(10)
);
```

`dim_date` has a similar use as `dim_location` and brings the same PROs and CONs.


```
CREATE TABLE dim_customer (
    customer_sk UUID PRIMARY KEY NOT NULL,
    customer_id UUID NOT NULL,
    full_name VARCHAR(255),
    email VARCHAR(255),
    address VARCHAR(255),
    mobile VARCHAR(20),
    registration_date TIMESTAMP,
    last_login TIMESTAMP,
    tier VARCHAR(50),
    valid_from DATETIME,
    valid_to DATETIME,
    is_current BOOLEAN
);
```

`dim_customer` is built usign a surrogate key because we are going to run SCD type 2 updates to the table. So it could happen that the same customer (so with a fixed customer_id), might have multiple records so we need a surrogate key to keep track of the new rows. On top of it, having a surrogate key helps us decoupling from the sourcing data and let us manage better the relationships.

Lets analyze some key takeaways of this schemas and what shall we take in account:
1. varchar for string variables (such as ip_address, city, state, long, lat, mobile, etc.) shall have the right size to have an optimized ddl
2. Our fact table relies on `customer_id` for join operations, but it may be that we have multiple records. It is then important to specify which record to pick using `is_current`.
